In [ ]:
import os, sys, argparse
from dataclasses import dataclass
from typing import Dict
from collections import deque, defaultdict
import numpy as np
import pygame
from pathlib import Path
import sys
import torch
from multiprocessing import Pool


In [ ]:
# evo_birds.py — life-cycle policy + continuous generations + synthetic elevation

IN_NOTEBOOK = "ipykernel" in sys.modules or "JPY_PARENT_PID" in os.environ
IS_WINDOWS = (os.name == "nt")

# ===================== CLI (robust in Jupyter + terminal) =====================

def _safe_parse_args():
    parser = argparse.ArgumentParser(add_help=True)
    parser.add_argument("--backend", choices=["numpy","torch","mp"], default="numpy")
    parser.add_argument("--workers", type=int, default=max(1, os.cpu_count() or 1))
    parser.add_argument("--fast", action="store_true", help="start in fast-forward")
    parser.add_argument("--algo", choices=["q","sarsa","mc"], default="q",
                        help="learning algorithm ('q' or 'sarsa')")

    # Jupyter/VS Code injects extra arguments → ignore them
    if any("ipykernel" in arg or "jupyter" in arg for arg in sys.argv):
        args, _ = parser.parse_known_args([])
    else:
        args, _ = parser.parse_known_args()
    return args

import sys
sys.argv = ["notebook", "--algo", "q"]  # or "q", "mc", "sarsa"
args = _safe_parse_args()

USE_TORCH = (args.backend == "torch")
USE_MP    = (args.backend == "mp")
USE_SARSA = (args.algo == "sarsa")
USE_MC    = (args.algo == "mc")

if USE_TORCH:
    try:
        import torch
        TORCH_DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
    except Exception:
        USE_TORCH = False
        TORCH_DEVICE = "cpu"

# ===================== CONFIG =====================
WORLD_W = 1024
WORLD_H = 1024
REGION  = 64            # 16x16 regions
TILE = 28
VIEW_W_TILES = 64
VIEW_H_TILES = 40

ELEV_NOISE_OCTAVES = 4
ELEV_WATER_LEVEL   = 0.36   # below this = water/river
ELEV_RIVER_BIAS    = 0.15   # deepens a sinuous river band

FPS_RENDER = 60
FPS_LOGIC_BASE = 30
FAST_FWD_MULT = 6 if args.fast else 3

SEASONS = ["Summer","Monsoon","Winter"]
STEPS_PER_SEASON = 250
YEARS_PER_GENERATION = 1

MINIMAP_SIZE = 352
MIGR_PATH_BUFFER = 8000
DRAW_SAMPLE_CAP = 3500
BASE_SPAWN_DENSITY = 0.1
MIGRATE_BASE_P = 0.25

# Life-cycle parameters
MATURITY_AGE_STEPS = 350          # becomes "adult" after this many steps
INCUBATION_STEPS   = 180          # steps after mating to lay/raise a chick
OFFSPRING_ENERGY   = 55.0
OFFSPRING_HEALTH   = 55.0
OFFSPRING_HYDR     = 55.0
OFFSPRING_SATIETY  = 55.0
POP_BUFFER         = 400          # extra empty slots for continuous generations
COURT_RADIUS       = 3            # search radius (Manhattan) for a partner

# Species & colors
INITIAL_COUNTS = {
    "Amazilia tzacatl": 1,
    "Amazilia saucerottei": 1,
    "Adelomyia melanogenys": 1,
    "Collocalia esculenta": 1,
    "Rupornis magnirostris": 1,
}
SPECIES_LIST = list(INITIAL_COUNTS.keys())
SPECIES_COLOR = {
    "Amazilia tzacatl": (0,200,160),
    "Amazilia saucerottei": (0,180,120),
    "Adelomyia melanogenys": (20,220,200),
    "Collocalia esculenta": (160,100,255),
    "Rupornis magnirostris": (240,170,60),
}

# Habitats & resources
HABITATS = ["Forest","Shrubland","Woodland","Grassland","Wetland","Marine","Coastal","Rock","Riverine","Human Modified"]
HAB_COL = {
    "Forest": (34,139,34), "Shrubland": (154,205,50), "Woodland": (46,139,87),
    "Grassland": (189,183,107), "Wetland": (72,209,204), "Marine": (25,25,112),
    "Coastal": (65,105,225), "Rock": (112,128,144), "Riverine": (30,144,255),
    "Human Modified": (160,82,45),
}
RES_TYPES = ["Nectar","Insects","Seeds","Vertebrate"]
RES_COL = {"Nectar":(255,220,0), "Insects":(20,20,20), "Seeds":(130,95,40), "Vertebrate":(200,70,70)}

HABITAT_MIX = {
    "Forest":         dict(Nectar=0.30, Insects=0.40, Seeds=0.20, Vertebrate=0.10),
    "Shrubland":      dict(Nectar=0.25, Insects=0.35, Seeds=0.30, Vertebrate=0.10),
    "Woodland":       dict(Nectar=0.20, Insects=0.30, Seeds=0.40, Vertebrate=0.10),
    "Grassland":      dict(Nectar=0.10, Insects=0.40, Seeds=0.40, Vertebrate=0.10),
    "Wetland":        dict(Nectar=0.15, Insects=0.50, Seeds=0.15, Vertebrate=0.20),
    "Marine":         dict(Nectar=0.00, Insects=0.50, Seeds=0.10, Vertebrate=0.40),
    "Coastal":        dict(Nectar=0.05, Insects=0.45, Seeds=0.15, Vertebrate=0.35),
    "Rock":           dict(Nectar=0.05, Insects=0.25, Seeds=0.20, Vertebrate=0.50),
    "Riverine":       dict(Nectar=0.20, Insects=0.50, Seeds=0.10, Vertebrate=0.20),
    "Human Modified": dict(Nectar=0.15, Insects=0.35, Seeds=0.30, Vertebrate=0.20),
}
SEASON_MULT = {
    "Summer":  dict(Nectar=0.8,  Insects=0.9, Seeds=1.0, Vertebrate=1.0),
    "Monsoon": dict(Nectar=0.9,  Insects=1.2, Seeds=0.9, Vertebrate=1.0),
    "Winter":  dict(Nectar=0.6,  Insects=0.7, Seeds=1.1, Vertebrate=1.0),
}
SEASON_COST = {
    "Summer":  dict(move_energy=0.35, dehydration=0.9,  cold=0.0,  flood=0.0),
    "Monsoon": dict(move_energy=0.35, dehydration=0.7,  cold=0.0,  flood=1.0),
    "Winter":  dict(move_energy=0.40, dehydration=0.7,  cold=0.80, flood=0.0),
}

# Tiles
T_FLOOR, T_WATER, T_ROCK, T_NEST, T_DESERT, T_TREE = range(6)

# ===================== Traits =====================
@dataclass
class Traits:
    mass: float; culmen: float; nares: float; width: float; depth: float; wing: float; hwi: float; tarsus: float
    @staticmethod
    def from_species() -> Dict[str,"Traits"]:
        return {
            "Rupornis magnirostris": Traits(269.0,26.4,17.4,10.4,13.4,220.9,28.0,61.5),
            "Collocalia esculenta":  Traits(6.3,7.1,2.5,1.7,1.6,99.5,68.3,8.1),
            "Amazilia tzacatl":      Traits(4.8,21.9,18.9,3.1,2.4,56.4,61.7,5.7),
            "Amazilia saucerottei":  Traits(4.5,20.8,16.4,2.7,2.2,51.7,65.1,4.9),
            "Adelomyia melanogenys": Traits(4.9,17.9,13.8,2.6,2.1,53.1,60.2,6.0),
        }
        

def _scale(x, lo, hi): 
    return np.clip((x-lo)/(hi-lo+1e-9), 0.0, 1.0)

def _effs_from_traits(tr: Traits):
    slender = tr.culmen / max(1e-6, tr.width+tr.depth)
    nectar = float(_scale(slender, 1.5, 10))
    seed   = float(_scale(tr.width+tr.depth, 3, 25))
    insect = float(np.clip(0.5*_scale(tr.wing, 40, 230) + 0.5*_scale(tr.hwi, 28, 70), 0, 1))
    verteb = float(np.clip(0.6*_scale(tr.mass, 4, 300) + 0.4*seed, 0, 1))
    cold   = float(_scale(tr.mass, 4, 300))
    heat   = float(1.0 - _scale(tr.mass, 4, 300))
    disp   = float(np.clip(0.5*_scale(tr.hwi,28,70)+0.5*_scale(tr.wing,40,230)-0.2*_scale(tr.mass,4,300),0,1))
    return nectar, seed, insect, verteb, cold, heat, disp

def mutate_traits(tr: Traits, rng: np.random.Generator) -> Traits:
    j=lambda x,p: max(1e-3, x*(1.0 + rng.normal(0, p)))
    return Traits(j(tr.mass,0.05), j(tr.culmen,0.03), j(tr.nares,0.03), j(tr.width,0.04),
                  j(tr.depth,0.04), j(tr.wing,0.03), j(tr.hwi,0.03), j(tr.tarsus,0.04))

def _fbm_noise(rng, h, w, octaves=4):
    """
    Fractal Brownian Motion (FBM) from summed blurred random fields.
    Returns float32 array in [0,1]. Keeps shape strictly (h, w).
    """
    def _blur3(a: np.ndarray) -> np.ndarray:
        # edge-clamped 3-tap box blur, separable, shape-preserving
        v = (np.pad(a, ((1,1),(0,0)), mode="edge")[0:-2, :] + a +
             np.pad(a, ((1,1),(0,0)), mode="edge")[2:, :]) / 3.0
        h = (np.pad(v, ((0,0),(1,1)), mode="edge")[:, 0:-2] + v +
             np.pad(v, ((0,0),(1,1)), mode="edge")[:, 2:]) / 3.0
        return h

    out  = np.zeros((h, w), dtype=np.float32)
    amp  = 0.5
    cur  = rng.random((h, w)).astype(np.float32)

    for _ in range(octaves):
        cur = _blur3(cur)           # smooth without changing size
        out += amp * cur
        amp *= 0.5
        cur = rng.random((h, w)).astype(np.float32)  # new octave seed

    out -= out.min()
    out /= (out.max() + 1e-9)
    return out


# ===================== World (global maps) =====================
class World:
    def __init__(self, rng: np.random.Generator):
        self.rng=rng
        self.reg_rows=WORLD_H//REGION; self.reg_cols=WORLD_W//REGION
        self.hab_grid = self._gen_habitat_grid()

        # --- synthetic elevation (0..1), with a sinuous river band downcut
        self.elev = _fbm_noise(rng, WORLD_H, WORLD_W, octaves=ELEV_NOISE_OCTAVES)
        yy, xx = np.mgrid[0:WORLD_H, 0:WORLD_W]
        river_band = 0.5 + 0.5*np.sin(xx / 32.0) * np.cos(yy / 64.0)
        self.elev = np.clip(self.elev - ELEV_RIVER_BIAS * (river_band > 0.55).astype(np.float32), 0.0, 1.0)

        # global tile map + per-resource global masks (uint8)
        self.tiles = np.full((WORLD_H, WORLD_W), T_FLOOR, dtype=np.uint8)
        self.resources = {k: np.zeros((WORLD_H, WORLD_W), dtype=np.uint8) for k in RES_TYPES}
        self.built_nest = np.zeros((WORLD_H, WORLD_W), dtype=np.uint8)

        self._carve_all()
        self.refresh_resources("Summer", parallel=(not IN_NOTEBOOK and not IS_WINDOWS and args.workers > 1))


    def _gen_habitat_grid(self):
        gr,gc=self.reg_rows,self.reg_cols
        noise=self.rng.random((gr,gc)); H=np.empty((gr,gc),dtype=object)
        for r in range(gr):
            for c in range(gc):
                v=noise[r,c]
                if c<2:              hb="Coastal" if r>2 else "Marine"
                elif r in (4,5,6):   hb="Riverine" if (c%3!=0) else "Wetland"
                elif v<0.15:         hb="Rock"
                elif v<0.35:         hb="Grassland"
                elif v<0.55:         hb="Shrubland"
                elif v<0.75:         hb="Woodland"
                else:                hb="Forest"
                if r>=gr-2 and v<0.3: hb="Human Modified"
                H[r,c]=hb
        return H

    def _carve_all(self):
        # Elevation-aware terrain carving
        t = np.full((WORLD_H, WORLD_W), T_FLOOR, np.uint8)

        # Water from low elevation
        water_mask = (self.elev <= ELEV_WATER_LEVEL)
        t[water_mask] = T_WATER

        # Rock on high elevation
        high = self.elev >= 0.80
        t[high] = T_ROCK

        # Region-specific features
        for gr in range(self.reg_rows):
            for gc in range(self.reg_cols):
                r0=gr*REGION; c0=gc*REGION
                sl = (slice(r0, r0+REGION), slice(c0, c0+REGION))
                hb = self.hab_grid[gr, gc]
                zon = t[sl]; elz = self.elev[sl]
                z = self.rng.random(zon.shape)

                if hb in ("Forest","Woodland","Shrubland"):
                    mask = (zon==T_FLOOR) & (elz>0.45) & (z<0.18)
                    zon[mask] = T_TREE

                if hb in ("Grassland","Rock","Human Modified"):
                    mask = (zon==T_FLOOR) & (elz>0.55) & (z<0.08)
                    zon[mask] = T_DESERT

                p = 0.05 if hb in ("Forest","Woodland") else 0.03
                mask = (zon!=T_WATER) & (zon!=T_ROCK) & (z<p)
                zon[mask] = T_NEST

                t[sl] = zon

        # keep map border walkable
        t[0,:]=T_FLOOR; t[-1,:]=T_FLOOR; t[:,0]=T_FLOOR; t[:,-1]=T_FLOOR
        self.tiles = t

    def refresh_resources(self, season: str, parallel=False):
        # clear
        for k in RES_TYPES:
            self.resources[k][:] = 0
    
        # Only parallelize on non-Windows, non-notebook runs
        use_parallel = bool(parallel and not IN_NOTEBOOK and not IS_WINDOWS and args.workers > 1)
    
        if use_parallel:
            # define a top-level helper to be picklable
            def _worker(args_tuple):
                gr, gc, REGION_, RES_TYPES_, BASE_SPAWN_DENSITY_, hab, mult_map, tiles_slice = args_tuple
                rng = np.random.default_rng((gr+1)*10007 + (gc+1)*7919)
                t = tiles_slice
                valid = np.ones_like(t, dtype=bool)
                if hab in ("Marine", "Coastal"):
                    valid &= (t == T_WATER)
                if hab in ("Desert",):
                    valid &= (t != T_WATER)
                free = np.argwhere((t != T_ROCK) & valid)
                if free.size == 0:
                    return (gr,gc,[])
                rng.shuffle(free)
                mix = HABITAT_MIX[hab]; mult = SEASON_MULT[season]
                weights = np.array([mix[r]*mult[r] for r in RES_TYPES_], float)
                weights /= weights.sum() + 1e-9
                n = int(len(free)*BASE_SPAWN_DENSITY_)
                outputs = []
                count=0; idx=0
                while count<n and idx<len(free):
                    rr,cc = free[idx]; idx+=1
                    name = RES_TYPES_[rng.choice(len(RES_TYPES_), p=weights)]
                    rt = t[rr,cc]
                    if name=="Nectar" and rt not in (T_TREE, T_FLOOR):
                        continue
                    if name=="Insects" and rt==T_WATER:
                        continue
                    if name=="Seeds" and rt==T_WATER:
                        continue
                    if name=="Vertebrate" and rt not in (T_WATER, T_TREE, T_ROCK):
                        continue
                    outputs.append((name, int(rr), int(cc)))
                    count+=1
                return (gr,gc,outputs)
    
            # build jobs
            jobs=[]
            for gr in range(self.reg_rows):
                for gc in range(self.reg_cols):
                    r0=gr*REGION; c0=gc*REGION
                    tiles_slice = self.tiles[r0:r0+REGION, c0:c0+REGION]
                    jobs.append((gr,gc,REGION,RES_TYPES,BASE_SPAWN_DENSITY,self.hab_grid[gr,gc],SEASON_MULT,tiles_slice))
    
            from multiprocessing import Pool
            with Pool(args.workers) as pool:
                results = pool.map(_worker, jobs)
    
            # write back
            for gr,gc,outs in results:
                r0=gr*REGION; c0=gc*REGION
                for name, rr, cc in outs:
                    self.resources[name][r0+rr, c0+cc] = 1
    
        else:
            # serial path
            rng = self.rng
            for gr in range(self.reg_rows):
                for gc in range(self.reg_cols):
                    r0=gr*REGION; c0=gc*REGION
                    t = self.tiles[r0:r0+REGION, c0:c0+REGION]
                    hab = self.hab_grid[gr,gc]
                    valid = np.ones_like(t, dtype=bool)
                    if hab in ("Marine", "Coastal"):
                        valid &= (t == T_WATER)
                    if hab in ("Desert",):
                        valid &= (t != T_WATER)
                    free = np.argwhere((t != T_ROCK) & valid)
                    if free.size == 0:
                        continue
                    rng.shuffle(free)
                    mix = HABITAT_MIX[hab]; mult=SEASON_MULT[season]
                    weights = np.array([mix[r]*mult[r] for r in RES_TYPES], float)
                    weights /= weights.sum()+1e-9
                    n = int(len(free)*BASE_SPAWN_DENSITY)
                    count=0; idx=0
                    while count<n and idx<len(free):
                        rr,cc = free[idx]; idx+=1
                        name = RES_TYPES[rng.choice(len(RES_TYPES), p=weights)]
                        rt = t[rr,cc]
                        if name=="Nectar" and rt not in (T_TREE, T_FLOOR):
                            continue
                        if name=="Insects" and rt==T_WATER:
                            continue
                        if name=="Seeds" and rt==T_WATER:
                            continue
                        if name=="Vertebrate" and rt not in (T_WATER, T_TREE, T_ROCK):
                            continue
                        self.resources[name][r0+rr, c0+cc]=1
                        count+=1


        # Final cleanup to ensure surface resources never sit on open water tiles
        water_mask = (self.tiles == T_WATER)
        for name in ("Nectar", "Insects", "Seeds"):
            self.resources[name][water_mask] = 0

    def habitat_at(self, r:int, c:int) -> str:
        return self.hab_grid[r//REGION, c//REGION]

# ===================== Population (array state) =====================
class Pop:
    def __init__(self, world: World, rng: np.random.Generator):
        self.world = world; self.rng=rng
        self.N0 = sum(INITIAL_COUNTS.values())     # initial alive
        self.N  = self.N0 + POP_BUFFER             # capacity

        # positions
        origin_r, origin_c = WORLD_H//2, WORLD_W//2
        self.r = np.full(self.N, origin_r, np.int32)
        self.c = np.full(self.N, origin_c, np.int32)

        jitter_r = np.clip(origin_r + rng.integers(-8,8,size=self.N0), 0, WORLD_H-1)
        jitter_c = np.clip(origin_c + rng.integers(-8,8,size=self.N0), 0, WORLD_W-1)
        self.r[:self.N0] = jitter_r.astype(np.int32)
        self.c[:self.N0] = jitter_c.astype(np.int32)

        # life state
        self.energy    = np.zeros(self.N, np.float32); self.energy[:self.N0]=90.0
        self.health    = np.zeros(self.N, np.float32); self.health[:self.N0]=90.0
        self.hydration = np.zeros(self.N, np.float32); self.hydration[:self.N0]=90.0
        self.satiety   = np.zeros(self.N, np.float32); self.satiety[:self.N0]=90.0
        self.twigs     = np.zeros(self.N, np.int16)
        self.nest_q    = np.zeros(self.N, np.float32)
        self.has_nest  = np.zeros(self.N, np.uint8)
        self.has_mated = np.zeros(self.N, np.uint8)
        self.alive     = np.zeros(self.N, np.uint8); self.alive[:self.N0]=1
        self.offspring_viability = np.zeros(self.N, np.float32)
        self.age       = np.zeros(self.N, np.int32)
        self.lifespan  = np.zeros(self.N, np.int32)

        # sex & reproduction
        self.sex       = np.zeros(self.N, np.uint8)   # 0=male,1=female
        self.sex[:self.N0] = rng.integers(0,2,size=self.N0)
        self.mature    = np.zeros(self.N, np.uint8)
        self.incubate  = np.full(self.N, -1, np.int32)  # -1=not incubating, else countdown

        # species index per bird
        sp_idx=[]
        for sp, n in INITIAL_COUNTS.items():
            sp_idx += [SPECIES_LIST.index(sp)]*n
        self.species_idx = np.zeros(self.N, np.int32)
        self.species_idx[:self.N0] = np.array(sp_idx, np.int32)

        # per-species effs
        self.nec_eff = np.zeros(self.N, np.float32)
        self.seed_eff= np.zeros(self.N, np.float32)
        self.ins_eff = np.zeros(self.N, np.float32)
        self.ver_eff = np.zeros(self.N, np.float32)
        self.cold_tol= np.zeros(self.N, np.float32)
        self.heat_tol= np.zeros(self.N, np.float32)
        self.dispersal = np.zeros(self.N, np.float32)
        self._init_traits()

        # linear Q weights
        self.W = np.zeros((11,10), np.float32)
        self.eps = 0.08; self.alpha=0.12; self.gamma=0.96

    def _init_traits(self):
        trait_map = Traits.from_species()
        start=0
        for sp in SPECIES_LIST:
            count = list(INITIAL_COUNTS.values())[SPECIES_LIST.index(sp)]
            T = trait_map[sp]
            nec,seed,ins,ver,cold,heat,disp = _effs_from_traits(T)
            sl = slice(start, start+count)
            self.nec_eff[sl]=nec; self.seed_eff[sl]=seed; self.ins_eff[sl]=ins; self.ver_eff[sl]=ver
            self.cold_tol[sl]=cold; self.heat_tol[sl]=heat; self.dispersal[sl]=disp
            start += count
        # ensure [0..N0) filled; rest zeros already

    def reinit_next_gen(self):
        origin_r, origin_c = WORLD_H//2, WORLD_W//2
        self.r[:] = origin_r
        self.c[:] = origin_c
        jitter_r = np.clip(origin_r + self.rng.integers(-8,8,size=self.N0), 0, WORLD_H-1)
        jitter_c = np.clip(origin_c + self.rng.integers(-8,8,size=self.N0), 0, WORLD_W-1)
        self.r[:self.N0] = jitter_r.astype(np.int32)
        self.c[:self.N0] = jitter_c.astype(np.int32)

        self.energy.fill(0); self.energy[:self.N0] = 90.0
        self.health.fill(0); self.health[:self.N0] = 90.0
        self.hydration.fill(0); self.hydration[:self.N0] = 90.0
        self.satiety.fill(0); self.satiety[:self.N0] = 90.0
        self.twigs.fill(0)
        self.nest_q.fill(0)
        self.has_nest.fill(0)
        self.has_mated.fill(0)
        self.alive.fill(0); self.alive[:self.N0] = 1
        self.offspring_viability.fill(0)
        self.age.fill(0)
        self.lifespan.fill(0)
        self.sex[:self.N0] = self.rng.integers(0,2,size=self.N0)
        self.sex[self.N0:] = 0
        self.mature.fill(0)
        self.incubate.fill(-1)

        # re-seed efficiencies with base traits
        self._init_traits()

    def spawn_offspring(self, parent_idx: int):
        # find free slot
        free = np.where(self.alive==0)[0]
        if free.size==0:
            return False
        slot = free[0]

        # inherit species & mutate parent's efficiencies slightly
        self.species_idx[slot] = self.species_idx[parent_idx]
        noise = lambda p: float(1.0 + self.rng.normal(0, p))
        def mclip(x, p=0.05): return float(np.clip(x*noise(p), 0.0, 1.0))

        self.nec_eff[slot] = mclip(self.nec_eff[parent_idx], 0.06)
        self.seed_eff[slot]= mclip(self.seed_eff[parent_idx], 0.06)
        self.ins_eff[slot] = mclip(self.ins_eff[parent_idx], 0.06)
        self.ver_eff[slot] = mclip(self.ver_eff[parent_idx], 0.06)
        self.cold_tol[slot]= mclip(self.cold_tol[parent_idx], 0.05)
        self.heat_tol[slot]= mclip(self.heat_tol[parent_idx], 0.05)
        self.dispersal[slot]=mclip(self.dispersal[parent_idx], 0.05)

        # place at parent location
        self.r[slot] = self.r[parent_idx]
        self.c[slot] = self.c[parent_idx]

        # init life stats
        self.energy[slot] = OFFSPRING_ENERGY
        self.health[slot] = OFFSPRING_HEALTH
        self.hydration[slot] = OFFSPRING_HYDR
        self.satiety[slot] = OFFSPRING_SATIETY
        self.twigs[slot] = 0
        self.nest_q[slot] = 0.0
        self.has_nest[slot] = 0
        self.has_mated[slot] = 0
        self.alive[slot] = 1
        self.offspring_viability[slot] = 0.0
        self.age[slot] = 0
        self.lifespan[slot] = 0
        self.sex[slot] = self.rng.integers(0,2)  # random sex
        self.mature[slot] = 0
        self.incubate[slot] = -1
        return True

# ===================== Backend steppers =====================
def _meta_policy_action(world: World, pop: Pop, season_idx: int):
    """
    Life-cycle meta-policy:
    Priority:
      1) Emergency water/food/health
      2) Build/secure nest (shelter)
      3) Court if mature + nest
      4) Otherwise move/explore
    Returns an action vector 'a_pref' with -1 for "no override".
    """
    N = pop.N
    a_pref = np.full(N, -1, np.int32)
    alive = (pop.alive>0)
    if not alive.any():
        return a_pref

    rr = pop.r; cc = pop.c
    tiles_here = world.tiles[rr, cc]
    season = SEASONS[season_idx]

    # Emergency thresholds
    low_hydr = (pop.hydration < 25)
    low_food = (pop.satiety   < 30)
    low_hp   = (pop.health    < 25)

    # If on water and thirsty: DRINK
    at_water = (tiles_here==T_WATER)
    a_pref[alive & low_hydr & at_water] = 5  # drink

    # If food resource right here & hungry: FORAGE
    here_food = (world.resources["Nectar"][rr,cc]>0) | (world.resources["Insects"][rr,cc]>0) | \
                (world.resources["Seeds"][rr,cc]>0) | (world.resources["Vertebrate"][rr,cc]>0)
    a_pref[alive & low_food & here_food] = 4  # forage

    # If weak: REST sometimes
    a_pref[alive & low_hp & (~at_water) & (~here_food)] = 9

    # Shelter/nest building if no nest: gather twigs or build at nest tile
    need_shelter = alive & (pop.has_nest==0) & (pop.mature>0)
    on_nest = (tiles_here==T_NEST)
    has_twigs = (pop.twigs>0)

    a_pref[need_shelter & on_nest & has_twigs] = 7  # build
    a_pref[need_shelter & (~on_nest)] = 6          # gather (fallback, simple)

    # Court if adult, has nest, decent condition
    cond_ok = (0.25*(pop.health+pop.energy+pop.hydration+pop.satiety) > 120.0)  # avg > 30
    ready_mate = alive & (pop.mature>0) & (pop.has_nest>0) & cond_ok & (pop.has_mated==0) & (pop.incubate<0)
    a_pref[ready_mate] = 8

    # Otherwise leave as -1 to let RL choose.
    return a_pref

def _find_local_partner(pop: Pop, idx: np.ndarray):
    """
    For each index in idx (adults trying to court), find a nearby opposite-sex partner
    within COURT_RADIUS who is also alive and not incubating.
    Returns boolean vector 'hit' and partner indices 'mate_idx' (or -1).
    """
    N = idx.size
    hit = np.zeros(N, np.bool_)
    mate_idx = np.full(N, -1, np.int32)
    if N==0:
        return hit, mate_idx

    for k, i in enumerate(idx):
        if pop.alive[i]==0 or pop.incubate[i]>=0:
            continue
        # search a small neighborhood
        ri, ci = int(pop.r[i]), int(pop.c[i])
        sex_need = 1 - int(pop.sex[i])  # opposite
        # candidates
        # restrict to alive, opposite sex, adult, not incubating, not already mated this season
        cand = np.where(
            (pop.alive>0) &
            (pop.sex==sex_need) &
            (pop.mature>0) &
            (pop.incubate<0) &
            (pop.has_mated==0)
        )[0]
        if cand.size==0:
            continue
        # distance filter
        dr = np.abs(pop.r[cand] - ri)
        dc = np.abs(pop.c[cand] - ci)
        near = cand[(dr + dc) <= COURT_RADIUS]
        if near.size==0:
            continue
        j = near[np.random.randint(0, near.size)]
        hit[k] = True
        mate_idx[k] = j
    return hit, mate_idx

def step_numpy(world: World, pop: Pop, season_idx: int, time_in_season: int):
    alive_mask = pop.alive.astype(bool)
    if not alive_mask.any():
        return {"avg_reward": 0.0, "avg_viability": 0.0, "survival_scores": {sp: 0.0 for sp in SPECIES_LIST}}

    season = SEASONS[season_idx]
    sc = SEASON_COST[season]
    move_pen = sc["move_energy"]
    dehydration = (0.5 if season=="Summer" else 0.4) * sc["dehydration"]
    cold_term = 0.6*sc["cold"]*(1.0 - pop.cold_tol)

    tiles_here = world.tiles[pop.r, pop.c]
    phi = np.zeros((pop.N, 11), np.float32)
    phi[np.arange(pop.N), tiles_here] = 1.0
    phi[:, 6 + season_idx] = 1.0
    phi[:, 9] = (pop.has_nest>0).astype(np.float32)
    phi[:,10] = (pop.has_mated>0).astype(np.float32)

    q = phi @ pop.W

    # ===== Life-cycle meta-policy override =====
    a = np.argmax(q, axis=1)
    a_pref = _meta_policy_action(world, pop, season_idx)
    rnd = np.random.random(pop.N) < pop.eps
    # apply overrides where set, else ε-greedy exploration
    use_pref = (a_pref>=0)
    a[use_pref] = a_pref[use_pref]
    a[~use_pref & rnd] = np.random.randint(0,10, size=(~use_pref & rnd).sum())
    # ===========================================

    # move deltas: 0..3 = NESW, 4 forage, 5 drink, 6 gather, 7 build, 8 court, 9 rest
    d = np.array([[-1,0],[0,1],[1,0],[0,-1],[0,0],[0,0],[0,0],[0,0],[0,0],[0,0]], np.int32)
    dr = d[a,0]; dc = d[a,1]
    nr = np.clip(pop.r + dr, 0, WORLD_H-1)
    nc = np.clip(pop.c + dc, 0, WORLD_W-1)
    tgt_tiles = world.tiles[nr, nc]
    blocked = (tgt_tiles == T_ROCK)
    nr[blocked] = pop.r[blocked]; nc[blocked] = pop.c[blocked]; tgt_tiles[blocked] = tiles_here[blocked]

    # --- elevation/water aware costs ---
    here_elev = world.elev[pop.r, pop.c]
    next_elev = world.elev[nr, nc]
    slope = np.abs(next_elev - here_elev)
    in_water_next = (tgt_tiles == T_WATER)

    desert_pen = np.where(tgt_tiles==T_DESERT, 0.4, 0.0)
    water_pen_e = np.where(in_water_next, 0.25, 0.0)
    slope_pen_e = 0.18 * slope
    pop.energy    = np.clip(pop.energy - move_pen - desert_pen - water_pen_e - slope_pen_e, 0, None)
    # -----------------------------------

    pop.satiety   = np.clip(pop.satiety - 0.35, 0, None)
    pop.hydration = np.clip(pop.hydration - dehydration, 0, None)
    water_cold = np.where(in_water_next, 0.20 * (0.5 if season=="Summer" else 1.0), 0.0)
    pop.health    = np.clip(pop.health - cold_term - water_cold, 0, None)

    pop.r = nr; pop.c = nc
    reward = np.full(pop.N, -0.05, np.float32)
    rr = pop.r; cc = pop.c

    # forage
    mask_forage = (a==4) & alive_mask
    if mask_forage.any():
        idx = np.where(mask_forage)[0]
        hasN = world.resources["Nectar"][rr[idx], cc[idx]]>0
        hasI = world.resources["Insects"][rr[idx], cc[idx]]>0
        hasS = world.resources["Seeds"][rr[idx],   cc[idx]]>0
        hasV = world.resources["Vertebrate"][rr[idx], cc[idx]]>0
        vals = np.stack([
            10*pop.nec_eff[idx]*hasN,
            10*pop.ins_eff[idx]*hasI,
            10*pop.seed_eff[idx]*hasS,
            10*pop.ver_eff[idx]*hasV
        ], axis=1)
        best = vals.argmax(axis=1)
        best_val = vals[np.arange(len(idx)), best]
        any_food = best_val>0
        if any_food.any():
            sel_idx = idx[any_food]
            sel_type = best[any_food]
            sel_val  = best_val[any_food]
            res_names = ["Nectar","Insects","Seeds","Vertebrate"]
            for k in range(4):
                k_mask = (sel_type==k)
                if not k_mask.any(): continue
                ii = sel_idx[k_mask]
                world.resources[res_names[k]][rr[ii], cc[ii]] = 0
            pop.satiety[sel_idx] = np.clip(pop.satiety[sel_idx] + 0.6*(sel_val), 0, 100)
            pop.energy[sel_idx]  = np.clip(pop.energy[sel_idx]  + 0.4*(sel_val), 0, 100)
            reward[sel_idx] += 0.6*(sel_val/10.0)
        reward[idx[~any_food]] -= 0.1

    # drink
    mask_drink = (a==5) & alive_mask
    if mask_drink.any():
        at_water = (world.tiles[rr[mask_drink], cc[mask_drink]]==T_WATER)
        succ = at_water | ((SEASONS[season_idx]=="Monsoon") & (np.random.random(at_water.shape[0])<0.05))
        if succ.any():
            ii = np.where(mask_drink)[0][succ]
            pop.hydration[ii] = np.clip(pop.hydration[ii] + 16, 0, 100)
            reward[ii] += 0.5
        jj = np.where(mask_drink)[0][~succ]
        reward[jj] -= 0.05

    # gather
    mask_gather = (a==6) & alive_mask
    if mask_gather.any():
        ok = (world.tiles[rr[mask_gather], cc[mask_gather]]!=T_ROCK)
        ii = np.where(mask_gather)[0][ok]
        pop.twigs[ii] += 1
        reward[ii] += 0.3
        jj = np.where(mask_gather)[0][~ok]
        reward[jj] -= 0.05
    # build
    mask_build = (a==7) & alive_mask
    if mask_build.any():
        at_nest = (world.tiles[rr[mask_build], cc[mask_build]]==T_NEST) & (pop.twigs[mask_build]>0)
        ii = np.where(mask_build)[0][at_nest]
        use = np.minimum(3, pop.twigs[ii])
        pop.twigs[ii] -= use
        pop.nest_q[ii] = np.clip(pop.nest_q[ii]+5*use, 0, 100)
        pop.has_nest[ii] = 1
        if ii.size>0:
            world.built_nest[rr[ii], cc[ii]] = 1
        reward[ii] += 1.0
        jj = np.where(mask_build)[0][~at_nest]
        reward[jj] -= 0.05
    # court (life-cycle mating)
    mask_court = (a==8) & alive_mask & (pop.mature>0) & (pop.has_nest>0) & (pop.incubate<0) & (pop.has_mated==0)
    if mask_court.any():
        ii = np.where(mask_court)[0]
        cond = (pop.health[ii]+pop.energy[ii]+pop.hydration[ii]+pop.satiety[ii])/(4*100.0)
        # search local partners
        hit, mates = _find_local_partner(pop, ii)
        if hit.any():
            sel = ii[hit]
            # success probability influenced by condition + nest quality + habitat flooding in monsoon
            nest = (pop.nest_q[sel]/100.0)
            flood = np.zeros_like(nest)
            if SEASONS[season_idx]=="Monsoon":
                habs = np.array([world.habitat_at(int(pop.r[k]), int(pop.c[k])) for k in sel], object)
                flood[np.isin(habs, ["Wetland","Coastal","Riverine"])] = 0.25
            p = np.clip((0.15 + 0.7*cond[hit] + 0.3*nest) * (1.0 - 0.5*flood), 0, 1)
            success = np.random.random(p.shape[0]) < p
            if success.any():
                sidx = sel[success]; midx = mates[hit][success]
                # mark both as mated; start incubation on the female
                pop.has_mated[sidx] = 1
                pop.has_mated[midx] = 1
                # start incubation timer on female (sex==1)
                fem = sidx[pop.sex[sidx]==1]
                fem2 = midx[pop.sex[midx]==1]
                for f in list(fem) + list(fem2):
                    pop.incubate[int(f)] = INCUBATION_STEPS
                reward[sidx] += 3.0
                reward[midx] += 3.0
            fail = ~success
            if fail.any():
                reward[sel[fail]] -= 0.2
        # those without nearby partner try and fail slightly
        no_partner = ~hit
        if no_partner.any():
            reward[ii[no_partner]] -= 0.1

    # rest
    mask_rest = (a==9) & alive_mask
    if mask_rest.any():
        ii = np.where(mask_rest)[0]
        torp = 1.0 + (1.0 - np.clip(1.0, 0, 1))  # ~1..2
        pop.health[ii] = np.clip(pop.health[ii] + 0.8*torp, 0, 100)
        pop.energy[ii] = np.clip(pop.energy[ii] - 0.4, 0, 100)

    # deaths
    died = (pop.energy==0) | (pop.hydration==0) | (pop.health==0)
    reward[died] -= 5.0
    pop.alive[died] = 0
    # natural maturity
    pop.mature[pop.age >= MATURITY_AGE_STEPS] = 1

    # offspring viability at end of Winter (kept as long-term shaped reward)
    season_end = (time_in_season == STEPS_PER_SEASON-1)
    if season_end and SEASONS[season_idx]=="Winter":
        ok = (pop.has_nest>0) & (pop.has_mated>0) & (pop.alive>0)
        if ok.any():
            ii = np.where(ok)[0]
            cond = (1.2*pop.health[ii] + pop.energy[ii] + pop.hydration[ii] + 0.8*pop.satiety[ii])
            habs = np.array([world.habitat_at(int(pop.r[k]), int(pop.c[k])) for k in ii], object)
            mixN = np.array([HABITAT_MIX[h]["Nectar"] for h in habs], np.float32)
            mixI = np.array([HABITAT_MIX[h]["Insects"] for h in habs], np.float32)
            mixS = np.array([HABITAT_MIX[h]["Seeds"] for h in habs], np.float32)
            mixV = np.array([HABITAT_MIX[h]["Vertebrate"] for h in habs], np.float32)
            eff = mixN*pop.nec_eff[ii] + mixI*pop.ins_eff[ii] + mixS*pop.seed_eff[ii] + mixV*pop.ver_eff[ii]
            v = (cond + 1.5*pop.nest_q[ii]) * (0.8 + 0.4*eff)
            v *= (0.9 + 0.2*pop.cold_tol[ii])
            v = np.clip(v, 0, None)
            pop.offspring_viability[ii] += v
            reward[ii] += 10.0*(1.0/(1.0 + np.exp(-0.04*(v-100))))

    # ===== Incubation countdown and birth (continuous generations) =====
    incub = np.where((pop.incubate>0) & (pop.alive>0))[0]
    pop.incubate[incub] -= 1
    birth_ready = np.where(pop.incubate==0)[0]
    for mom in birth_ready:
        # produce 1 chick (could randomize litter size if desired)
        ok = pop.spawn_offspring(mom)
        if ok:
            reward[mom] += 8.0  # big terminal-ish reward toward the true objective
        pop.incubate[mom] = -1
        pop.has_mated[mom] = 0
        # optional: slight stat drain after raising offspring
        pop.energy[mom] = max(0.0, pop.energy[mom]-6.0)
        pop.satiety[mom]= max(0.0, pop.satiety[mom]-6.0)
    # ================================================================

    # TD(0) linear update
    phi2 = np.zeros_like(phi)
    phi2[np.arange(pop.N), tgt_tiles] = 1.0
    phi2[:, 6 + season_idx] = 1.0
    phi2[:, 9] = (pop.has_nest>0).astype(np.float32)
    phi2[:,10] = (pop.has_mated>0).astype(np.float32)

    q_all = phi @ pop.W
    q_sa = q_all[np.arange(pop.N), a]
    q_next = phi2 @ pop.W
    if USE_SARSA:
        a_next = np.argmax(q_next, axis=1)
        a_pref_next = _meta_policy_action(world, pop, season_idx)
        rnd_next = np.random.random(pop.N) < pop.eps
        use_pref_next = (a_pref_next>=0)
        a_next[use_pref_next] = a_pref_next[use_pref_next]
        a_next[~use_pref_next & rnd_next] = np.random.randint(0,10, size=(~use_pref_next & rnd_next).sum())
        target = reward + pop.gamma * q_next[np.arange(pop.N), a_next]
    elif USE_MC:
        target = reward.copy()
    else:
        target = reward + pop.gamma * np.max(q_next, axis=1)
    target[died] = reward[died]

    for act in range(10):
        mask = (a==act) & alive_mask
        if not mask.any(): continue
        td = (target[mask] - q_sa[mask]).reshape(-1,1).astype(np.float32)
        pop.W[:,act] += pop.alpha * (phi[mask].T @ td).flatten()

    # migration (kept)
    if season_end:
        mig_p = MIGRATE_BASE_P * pop.dispersal
        do_mig = (np.random.random(pop.N) < mig_p) & (pop.alive>0)
        if do_mig.any():
            gr = pop.r//REGION; gc = pop.c//REGION
            choices = []
            for dr,dc in [(-1,0),(1,0),(0,-1),(0,1)]:
                ngr = np.clip(gr+dr, 0, world.reg_rows-1)
                ngc = np.clip(gc+dc, 0, world.reg_cols-1)
                choices.append((ngr,ngc))
            pick = np.random.randint(0,4,size=pop.N)
            trg_r = choices[0][0]*(pick==0) + choices[1][0]*(pick==1) + choices[2][0]*(pick==2) + choices[3][0]*(pick==3)
            trg_c = choices[0][1]*(pick==0) + choices[1][1]*(pick==1) + choices[2][1]*(pick==2) + choices[3][1]*(pick==3)
            rr = np.random.randint(1, REGION-1, size=pop.N)
            cc = np.random.randint(1, REGION-1, size=pop.N)
            new_r = trg_r*REGION + rr
            new_c = trg_c*REGION + cc
            pop.energy[do_mig] = np.clip(pop.energy[do_mig] - 2.5*(1.0 - pop.dispersal[do_mig]), 0, 100)
            pop.hydration[do_mig] = np.clip(pop.hydration[do_mig] - 2.0, 0, 100)
            pop.r[do_mig] = new_r[do_mig]; pop.c[do_mig] = new_c[do_mig]

    pop.age[pop.alive>0] += 1
    pop.lifespan[pop.alive>0] += 1

    avg_reward = float(reward.mean()) if reward.size else 0.0
    alive_now = (pop.alive>0)
    if alive_now.any():
        vitality = (pop.health[alive_now] + pop.energy[alive_now] + pop.hydration[alive_now] + pop.satiety[alive_now]) * 0.25
        avg_viability = float(np.mean(vitality))
    else:
        avg_viability = 0.0
    survival_scores = {}
    for sp_idx, sp in enumerate(SPECIES_LIST):
        mask = alive_now & (pop.species_idx==sp_idx)
        if mask.any():
            survival_scores[sp] = float(np.mean(pop.health[mask] + pop.energy[mask] + pop.hydration[mask] + pop.satiety[mask]))
        else:
            survival_scores[sp] = 0.0

    return {
        "avg_reward": avg_reward,
        "avg_viability": avg_viability,
        "survival_scores": survival_scores
    }

def step_torch(world: World, pop: Pop, season_idx: int, time_in_season: int):
    # Torch GPU port is possible by translating arrays to tensors; for now reuse NumPy vectorized step.
    return step_numpy(world, pop, season_idx, time_in_season)

def step_mp(world: World, pop: Pop, season_idx: int, time_in_season: int, workers: int):
    # NumPy already releases the GIL inside vector ops; mp is usually not necessary.
    return step_numpy(world, pop, season_idx, time_in_season)

# ===================== Simulator & Renderer =====================
class Sim:
    def __init__(self):
        self.rng=np.random.default_rng(123)
        self.world=World(self.rng)
        self.pop=Pop(self.world, self.rng)
        self.season_idx=0; self.time_in_season=0; self.year=0; self.generation=0
        self.fast_forward = args.fast
        self.migrations = deque(maxlen=MIGR_PATH_BUFFER)
        self.stats_deaths = defaultdict(int)
        self.focus_species_idx = -1
        self.follow_centroid = True
        self.hud_detail=True; self.show_migrations=True
        self.metrics_history = {
            "avg_reward": deque(maxlen=600),
            "avg_viability": deque(maxlen=600),
            "survival": {sp: deque(maxlen=600) for sp in SPECIES_LIST}
        }
        self.last_metrics = {}
        self.algo_name = "MC-Control" if USE_MC else ("SARSA" if USE_SARSA else "Q-Learning")

    def force_new_generation(self, reset_metrics=True, increment=True):
        if increment:
            self.generation += 1
        self.pop.reinit_next_gen()
        self.world.built_nest.fill(0)
        if reset_metrics:
            self.metrics_history["avg_reward"].clear()
            self.metrics_history["avg_viability"].clear()
            for hist in self.metrics_history["survival"].values():
                hist.clear()
        self.last_metrics = {}



    def _record_metrics(self, metrics):
        if not metrics:
            return
        self.last_metrics = metrics
        self.metrics_history["avg_reward"].append(metrics.get("avg_reward", 0.0))
        self.metrics_history["avg_viability"].append(metrics.get("avg_viability", 0.0))
        surv = metrics.get("survival_scores", {})
        for sp, hist in self.metrics_history["survival"].items():
            hist.append(surv.get(sp, 0.0))

    def curr_season(self): return SEASONS[self.season_idx]

    def step(self):
        if USE_TORCH:
            metrics = step_torch(self.world, self.pop, self.season_idx, self.time_in_season)
        elif USE_MP and args.workers>1:
            metrics = step_mp(self.world, self.pop, self.season_idx, self.time_in_season, args.workers)
        else:
            metrics = step_numpy(self.world, self.pop, self.season_idx, self.time_in_season)
        self._record_metrics(metrics)

        self.time_in_season += 1
        if self.time_in_season >= STEPS_PER_SEASON:
            self.time_in_season = 0
            self.season_idx = (self.season_idx+1) % 3
            if self.season_idx==0: self.year += 1
            self.world.refresh_resources(self.curr_season(), parallel=True)
            if self.curr_season()=="Monsoon":
                r= self.pop.r; c=self.pop.c
                habs = np.array([self.world.habitat_at(int(r[i]), int(c[i])) for i in range(self.pop.N)], object)
                wet = np.isin(habs, ["Wetland","Riverine","Coastal"])
                hit = wet & (np.random.random(self.pop.N) < 0.30)
                self.pop.has_nest[hit]=0; self.pop.nest_q[hit]=0
                if hit.any():
                    self.world.built_nest[r[hit], c[hit]] = 0
            # IMPORTANT: disable hard generation reset — continuous generations instead
            # if (self.year % YEARS_PER_GENERATION)==0 and self.season_idx==0:
            #     self.generation += 1
            #     self.pop.reinit_next_gen()
            # We still tick a generation counter slowly to visualize progress
            if self.season_idx==0:
                self.force_new_generation(reset_metrics=False)

class Renderer:
    def __init__(self, sim: Sim):
        pygame.init()
        flags = pygame.SCALED | pygame.RESIZABLE
        self.sim=sim; self.tile=TILE; self.view_w=VIEW_W_TILES; self.view_h=VIEW_H_TILES; self.hud_w=460
        self.screen = pygame.display.set_mode((self.view_w*self.tile + self.hud_w, self.view_h*self.tile), flags)
        pygame.display.set_caption(f"Evolutionary Birds — {('TORCH' if USE_TORCH else 'NUMPY' if not USE_MP else 'MP')} backend")
        self.clock=pygame.time.Clock()
        self.font=pygame.font.SysFont(None, 22); self.big=pygame.font.SysFont(None, 26); self.font_small=pygame.font.SysFont(None, 18)
        self.cam_x = np.clip(sim.pop.c[:sim.pop.N0].mean().astype(int) - self.view_w//2, 0, WORLD_W-self.view_w)
        self.cam_y = np.clip(sim.pop.r[:sim.pop.N0].mean().astype(int) - self.view_h//2, 0, WORLD_H-self.view_h)
        self.minimap=None; self.minimap_dirty=True
        self.last_logic_tick = pygame.time.get_ticks()
        self._icon_cache = {}
        self._load_icons()

    def _load_icons(self):
        try:
            base = Path(__file__).resolve().parent
        except NameError:
            base = Path.cwd()
            if (base / "Code files").exists():
                base = base / "Code files"
        icon_dir = base / "icons"
        def _load(name):
            if not name:
                return None
            try:
                return pygame.image.load(str(icon_dir / name)).convert_alpha()
            except Exception:
                return None
        bird_files = ["bird1.png", "bird2.png", "bird3.png", "bird4.png", "bird5.png"]
        self.species_icons = {sp: _load(fname) for sp, fname in zip(SPECIES_LIST, bird_files)}
        self.tile_icons = {
            T_FLOOR: None,
            T_WATER: _load("water1.png"),
            T_ROCK: _load("mountain1.png"),
            T_NEST: None,
            T_DESERT: _load("sand1.png"),
            T_TREE: _load("tree1.png"),
        }
        self.nest_icon = _load("nest2.png")
        self.resource_icons = {
            "Nectar": _load("tree2.png") or _load("tree1.png"),
            "Insects": _load("next1.png"),
            "Seeds": _load("nuts1.png"),
            "Vertebrate": _load("bird6.png"),
        }

    def _icon_surface(self, key, surface, size):
        if surface is None:
            return None
        size = int(max(1, size))
        cache_key = (key, size)
        surf = self._icon_cache.get(cache_key)
        if surf is None:
            surf = pygame.transform.smoothscale(surface, (size, size))
            self._icon_cache[cache_key] = surf
        return surf

    def run(self):
        running=True
        while running:
            running = self.handle_input()
            steps = self.logic_steps()
            for _ in range(steps):
                self.sim.step(); self.minimap_dirty=True
            if self.sim.follow_centroid: self.follow_center()
            self.draw()
            self.clock.tick(FPS_RENDER)

    def logic_steps(self):
        target = FPS_LOGIC_BASE * (FAST_FWD_MULT if self.sim.fast_forward else 1)
        now = pygame.time.get_ticks()
        dt = now - self.last_logic_tick
        steps = int(dt * target / 1000.0)
        if steps>0: self.last_logic_tick = now
        return steps

    def handle_input(self):
        for e in pygame.event.get():
            if e.type==pygame.QUIT: return False
            if e.type==pygame.KEYDOWN:
                if e.key in (pygame.K_ESCAPE, pygame.K_q): return False
                if e.key==pygame.K_SPACE: self.sim.fast_forward = not self.sim.fast_forward
                if e.key==pygame.K_h: self.sim.hud_detail = not self.sim.hud_detail
                if e.key==pygame.K_m: self.sim.show_migrations = not self.sim.show_migrations
                if e.key==pygame.K_n: self.sim.season_idx=(self.sim.season_idx+1)%3; self.sim.world.refresh_resources(self.sim.curr_season(), parallel=True)
                if e.key==pygame.K_c: self.sim.follow_centroid = not self.sim.follow_centroid
                if e.key in (pygame.K_EQUALS, pygame.K_PLUS): self.zoom(+2)
                if e.key==pygame.K_MINUS: self.zoom(-2)
                if e.key==pygame.K_g:
                    self.sim.force_new_generation(reset_metrics=True)
                    self.minimap_dirty = True
                if pygame.K_0 <= e.key <= pygame.K_5:
                    idx = e.key - pygame.K_1
                    self.sim.focus_species_idx = -1 if e.key==pygame.K_0 else max(0, min(len(SPECIES_LIST)-1, idx))
        keys=pygame.key.get_pressed(); pan=10
        if keys[pygame.K_LEFT] or keys[pygame.K_a]: self.cam_x-=pan
        if keys[pygame.K_RIGHT]or keys[pygame.K_d]: self.cam_x+=pan
        if keys[pygame.K_UP]   or keys[pygame.K_w]: self.cam_y-=pan
        if keys[pygame.K_DOWN] or keys[pygame.K_s]: self.cam_y+=pan
        self.cam_x=np.clip(self.cam_x, 0, WORLD_W-self.view_w)
        self.cam_y=np.clip(self.cam_y, 0, WORLD_H-self.view_h)
        return True

    def zoom(self, delta):
        prev = self.tile
        self.tile = int(np.clip(self.tile+delta, 14, 48))
        if self.tile != prev:
            self._icon_cache.clear()
        size = (self.view_w*self.tile + self.hud_w, self.view_h*self.tile)
        flags = pygame.SCALED | pygame.RESIZABLE
        try:
            self.screen = pygame.display.set_mode(size, flags)
        except pygame.error:
            pygame.display.quit()
            pygame.display.init()
            self.screen = pygame.display.set_mode(size)

    def follow_center(self):
        built_view = self.sim.world.built_nest[self.cam_y:self.cam_y+self.view_h, self.cam_x:self.cam_x+self.view_w]
        nest_surface = None
        if self.nest_icon is not None and np.any(built_view):
            nest_surface = self._icon_surface(("built-nest", self.tile), self.nest_icon, max(18, int(self.tile*0.9)))
        if nest_surface is not None:
            ys, xs = np.nonzero(built_view)
            for y,x in zip(ys,xs):
                cx = x*TILE + TILE//2
                cy = y*TILE + TILE//2
                rect = nest_surface.get_rect(center=(cx, cy))
                self.screen.blit(nest_surface, rect)
        pop=self.sim.pop
        if self.sim.focus_species_idx==-1:
            mask=(pop.alive>0)
        else:
            mask=(pop.alive>0) & (pop.species_idx==self.sim.focus_species_idx)
        if not mask.any(): return
        cy=int(pop.r[mask].mean()); cx=int(pop.c[mask].mean())
        self.cam_x=np.clip(cx - self.view_w//2, 0, WORLD_W-self.view_w)
        self.cam_y=np.clip(cy - self.view_h//2, 0, WORLD_H-self.view_h)

    def build_minimap(self):
        gr = WORLD_H//REGION; gc = WORLD_W//REGION
        surf=pygame.Surface((gc,gr))
        arr=pygame.PixelArray(surf)
        for r in range(gr):
            for c in range(gc):
                col=HAB_COL[self.sim.world.hab_grid[r,c]]
                arr[c,r]=surf.map_rgb(col)
        del arr
        base = pygame.transform.smoothscale(surf, (MINIMAP_SIZE, MINIMAP_SIZE))
        overlay=pygame.Surface((MINIMAP_SIZE,MINIMAP_SIZE), pygame.SRCALPHA)
        bins=np.zeros((gr,gc), np.int32)
        rr=(self.sim.pop.r//REGION).clip(0,gr-1)
        cc=(self.sim.pop.c//REGION).clip(0,gc-1)
        alive=(self.sim.pop.alive>0)
        np.add.at(bins, (rr[alive],cc[alive]), 1)
        m=bins.max() if bins.max()>0 else 1
        for r in range(gr):
            for c in range(gc):
                v=bins[r,c]/m
                if v<=0: continue
                col=(0,0,0,int(160*v))
                rx=int(c*MINIMAP_SIZE/gc); ry=int(r*MINIMAP_SIZE/gr)
                rw=int(MINIMAP_SIZE/gc)+1; rh=int(MINIMAP_SIZE/gr)+1
                pygame.draw.rect(overlay, col, (rx,ry,rw,rh))
        base.blit(overlay,(0,0))
        self.minimap=base; self.minimap_dirty=False

    def draw(self):
        if self.minimap is None or self.minimap_dirty: self.build_minimap()
        for ty in range(self.view_h):
            wy=self.cam_y+ty
            for tx in range(self.view_w):
                wx=self.cam_x+tx
                hb = self.sim.world.habitat_at(wy, wx)
                pygame.draw.rect(self.screen, HAB_COL[hb], (tx*TILE, ty*TILE, TILE, TILE))
        sub_tiles = self.sim.world.tiles[self.cam_y:self.cam_y+self.view_h, self.cam_x:self.cam_x+self.view_w]
        dot = max(3, TILE//10)
        for y in range(sub_tiles.shape[0]):
            for x in range(sub_tiles.shape[1]):
                t = sub_tiles[y,x]
                px=x*TILE; py=y*TILE
                icon = self.tile_icons.get(t)
                if icon is not None:
                    scaled = self._icon_surface(("tile", t), icon, max(14, int(self.tile*0.9)))
                    if scaled is not None:
                        rect = scaled.get_rect(center=(px+TILE//2, py+TILE//2))
                        self.screen.blit(scaled, rect)
        for name,col in RES_COL.items():
            mask = self.sim.world.resources[name][self.cam_y:self.cam_y+self.view_h, self.cam_x:self.cam_x+self.view_w]
            ys, xs = np.nonzero(mask)
            for y,x in zip(ys,xs):
                icon = self.resource_icons.get(name)
                if icon is not None:
                    scaled = self._icon_surface(("res", name), icon, max(12, int(self.tile*0.5)))
                    if scaled is not None:
                        rect = scaled.get_rect(center=(x*TILE+TILE//2, y*TILE+TILE//2))
                        self.screen.blit(scaled, rect)
                        continue
                pygame.draw.circle(self.screen, col, (x*TILE+TILE//2, y*TILE+TILE//2), dot)
        pop=self.sim.pop
        if self.sim.focus_species_idx==-1:
            mask = (pop.alive>0)
        else:
            mask = (pop.alive>0) & (pop.species_idx==self.sim.focus_species_idx)
        idx = np.where(mask & (pop.r>=self.cam_y) & (pop.r<self.cam_y+self.view_h) & (pop.c>=self.cam_x) & (pop.c<self.cam_x+self.view_w))[0]
        if idx.size > DRAW_SAMPLE_CAP: idx = idx[::max(1, idx.size//DRAW_SAMPLE_CAP)]
        radius=max(4, TILE//7)
        for i in idx:
            x = (pop.c[i]-self.cam_x)*TILE + TILE//2
            y = (pop.r[i]-self.cam_y)*TILE + TILE//2
            col = SPECIES_COLOR[SPECIES_LIST[pop.species_idx[i]]]
            pygame.draw.circle(self.screen, col, (x,y), radius)
            icon = self.species_icons.get(SPECIES_LIST[pop.species_idx[i]])
            if icon is not None:
                scaled = self._icon_surface(("species", SPECIES_LIST[pop.species_idx[i]]), icon, max(16, int(self.tile*0.85)))
                if scaled is not None:
                    rect = scaled.get_rect(center=(x,y))
                    self.screen.blit(scaled, rect)
        self.draw_hud()
        pygame.display.flip()

    def draw_hud(self):
        x = self.view_w*TILE + 10
        y = 10
        width = self.hud_w - 20
        height = self.view_h*TILE - 20
        panel = pygame.Surface((width, height), pygame.SRCALPHA)
        panel.fill((12, 12, 16, 225))
        self.screen.blit(panel, (x, y))
        pygame.draw.rect(self.screen, (200, 200, 210), (x, y, width, height), 1)

        text_x = x + 16
        text_y = y + 16
        self.screen.blit(self.minimap, (text_x, text_y))
        pygame.draw.rect(self.screen, (210, 210, 220), (text_x, text_y, MINIMAP_SIZE, MINIMAP_SIZE), 1)

        block_y = text_y + MINIMAP_SIZE + 18
        pop = self.sim.pop
        total = int((pop.alive>0).sum())
        info_lines = [
            f"Gen {self.sim.generation}  Year {self.sim.year}  Season {self.sim.curr_season()}",
            f"Population: {total}",
        ]
        for i, sp in enumerate(SPECIES_LIST):
            count = int(((pop.alive>0) & (pop.species_idx==i)).sum())
            info_lines.append(f"  {sp}: {count}")
        info_lines.extend([
            f"Focus: {'ALL' if self.sim.focus_species_idx==-1 else SPECIES_LIST[self.sim.focus_species_idx]}  Follow: {'ON' if self.sim.follow_centroid else 'OFF'}",
            f"Algo: {self.sim.algo_name}"
        ])
        for line in info_lines:
            self._text(line, text_x, block_y, font=self.font_small)
            block_y += 18

        block_y += 6
        graph_x = text_x
        graph_w = width - 32
        self._draw_line_graph(graph_x, block_y, graph_w, 64, self.sim.metrics_history["avg_reward"], (90, 170, 255), "Avg reward")
        block_y += 72
        self._draw_line_graph(graph_x, block_y, graph_w, 64, self.sim.metrics_history["avg_viability"], (220, 210, 120), "Avg vitality")
        block_y += 72
        self._draw_multi_graph(graph_x, block_y, graph_w, 124, self.sim.metrics_history["survival"])
        block_y += 134
        if self.sim.last_metrics:
            surv = self.sim.last_metrics.get("survival_scores", {})
            if surv:
                leader = max(surv.items(), key=lambda kv: kv[1])[0]
                self._text(f"Top survival: {leader}", graph_x, block_y, bold=True, font=self.font_small)
                block_y += 22

        block_y = max(block_y + 10, y + height - (len(RES_TYPES)*20 + 48))
        self._text("Resources", text_x, block_y, True, font=self.font_small)
        block_y += 22
        for rn, col in RES_COL.items():
            icon = self.resource_icons.get(rn)
            if icon is not None:
                scaled = self._icon_surface(("hud-res", rn), icon, 18)
                if scaled is not None:
                    self.screen.blit(scaled, (text_x, block_y-2))
            else:
                pygame.draw.circle(self.screen, col, (text_x+10, block_y+6), 6)
            self._text(rn, text_x+26, block_y-2, font=self.font_small)
            block_y += 20

    def _text(self, s, x, y, bold=False, font=None, color=(235,235,235)):
        font_obj = font or (self.big if bold else self.font)
        shadow = font_obj.render(s, True, (0,0,0))
        self.screen.blit(shadow, (x+1, y+1))
        self.screen.blit(font_obj.render(s, True, color), (x, y))

    def _draw_line_graph(self, x, y, width, height, history, color, title):
        rect = pygame.Rect(x, y, width, height)
        pygame.draw.rect(self.screen, (20,20,24), rect)
        pygame.draw.rect(self.screen, (70,70,80), rect, 1)
        values = list(history)
        if len(values) >= 2:
            pts, min_v, max_v = self._history_points(values, x, y, width, height)
            if pts:
                pygame.draw.lines(self.screen, color, False, pts, 2)
                base_y = y + height - 2
                pygame.draw.line(self.screen, (60,60,65), (x+1, base_y), (x+width-2, base_y), 1)
                label = f"min {min_v:.1f}  max {max_v:.1f}"
                self._text(label, x+6, y+height-18, font=self.font_small)
        self._text(title, x, y-18, True, font=self.font_small)

    def _draw_multi_graph(self, x, y, width, height, history_map):
        rect = pygame.Rect(x, y, width, height)
        pygame.draw.rect(self.screen, (18,18,22), rect)
        pygame.draw.rect(self.screen, (65,65,72), rect, 1)
        all_values = []
        for hist in history_map.values():
            all_values.extend(hist)
        if len(all_values) >= 2:
            _, min_v, max_v = self._history_points(all_values, x, y, width, height, normalize_only=True)
            span = (max_v - min_v) or 1.0
            for sp, hist in history_map.items():
                values = list(hist)
                if len(values) < 2:
                    continue
                pts = []
                for idx, val in enumerate(values):
                    px = x + 2 + int(idx/(len(values)-1)*(width-4))
                    py = y + height - 2 - int((val-min_v)/span*(height-4))
                    pts.append((px, py))
                if pts:
                    col = SPECIES_COLOR[sp]
                    pygame.draw.lines(self.screen, col, False, pts, 2)
            legend_x = x + width - 132
            legend_y = y + 8
            for sp in SPECIES_LIST:
                pygame.draw.rect(self.screen, SPECIES_COLOR[sp], (legend_x, legend_y, 12, 12))
                self._text(sp, legend_x + 18, legend_y-2, font=self.font_small)
                legend_y += 16
            self._text(f"Survival scores  min {min_v:.1f}  max {max_v:.1f}", x, y-18, True, font=self.font_small)
        else:
            self._text("Survival scores", x, y-18, True, font=self.font_small)

    def _history_points(self, values, x, y, width, height, normalize_only=False):
        if len(values) < 2:
            return ([], 0.0, 0.0) if not normalize_only else ([], 0.0, 0.0)
        min_v = min(values)
        max_v = max(values)
        if normalize_only:
            return ([], min_v, max_v)
        span = (max_v - min_v) or 1.0
        pts = []
        for idx, val in enumerate(values):
            px = x + 2 + int(idx/(len(values)-1)*(width-4))
            py = y + height - 2 - int((val-min_v)/span*(height-4))
            pts.append((px, py))
        return pts, min_v, max_v

# ===================== main =====================
def main():
    sim=Sim()
    Renderer(sim).run()

if __name__=="__main__":
    main()






pygame 2.6.1 (SDL 2.28.4, Python 3.10.9)
Hello from the pygame community. https://www.pygame.org/contribute.html


: 